# <span style='background:DarkOliveGreen'> The Trading Strategy using Moving Averages 9x21   🐍</span>

<strong> Introduction </strong>

The objective of this code is to be able to reduce the work of analysing a large number of stocks.
The idea is that the investor already have selected a list of stocks he wants to analyse, and he will use this code to decide which ones to buy, to hold or to stop ("close the position"). 
He can and should combine Fundamental Analysis and Technical Analysis.

He should use Fundamental Analysis to generate the list of companies he would like to invest on - this part is not developed as a code in this Jupyter NoteBook (JNB), but is briefly discussed below.

<strong> Fundamental Analysis </strong>

The idea is to get a list of companies that are solid and could be interesting to hold for some time.
In the beginning of the code, we provide a list of stocks traded in the Brazilian Market (called B3) based on the IDIV Index.
This index gathers stocks that pay good dividends and have good liquidity.  It's composition can be downloaded at B3 site:
https://www.b3.com.br/pt_br/market-data-e-indices/indices/indices-de-segmentos-e-setoriais/indice-dividendos-idiv-composicao-da-carteira.htm

Therefore, using the IDIV is a good way to select good companies and then we can use Technical Analysis to decide WHEN to: buy, hold or stop.


<strong> Technical Analysis </strong>

The list will be be tested by a classic Technical Indicator: the combination of two Moving Averages (MAs).
In this code I use the MAs of 9 and 21 days, as it is a classic.  But the code can be easily adapted for any other combination of MAs.

<strong> Why using MAs? </strong>

MAs are used to identify trends.  The idea is that if the "fast" one (9 periods) crosses the "slow" one (21 periods) upwards, the stock is in a bullish trend, so we should buy it; 
if it crosses downwards, we should sell it (in this case we will use it as a STOP sign, we should "close the position" (that is, sell all the stocks we have).
We could also open a short position on that specific stock, however we are not this risk-taker and, due to liquidity issues in the Brazilian market, that might be not a good idea.


Hence, the idea is that this code should be run daily. </p>
It will give you the list of stocks classified in 4 cases:
1. if MA9 crossed MA21 up on the previous day: buy
2. if MA9 is higher than MA21 but it was already higher on the day before: hold (if you already bought it before, otherwise wait)
3. if MA9 crossed MA21 down: sell (if you already bought it before, otherwise wait)
obs: If we sell above the price we bought, it is considered "Take Profit" (TP), if we sell below the price we paid, it is considered "Stop Loss" (SL).
4. if MA9 is lower than MA21 and it was already lower on the day before: wait (you should not have a stock in this classification in your portfolio)
(for more on this topic: https://www.daytradetheworld.com/trading-blog/alexander-elder-trade-market-beginners-2/#5_-_Create_a_money_Management_Plan)

As we can see, the movement of crossing is used as signal for buying or selling.
However, I think it should not be used alone, you should combine it with a few more technical indicators.
I use this code to reduce the number of stocks I will analyse - I open a full Graph of the stocks in situation i) and check a few other indicators, as the RSI, for example (I like to use TradingView website for it).  Then I decide to buy it or not.

However, it is strongly advised to use iii) as a STOP signal!

Stocks in situation ii) are more complicated - you can chose to use another criteria to make a partial TP.  That could be a RSI > 70, for example.
Stocks in situation iv): if you have time you can check on them, maybe one of them is "giving" a signal of reversing the tendency (for example: a hammer candle) that the criteria MA9x21 will not identify.  However, if you buy a stock classified on "wait" it is a riskier trade and you should define another criteria for SL - as MA9 is already below MA21 this code will not give you a SL signal.

In [25]:
# importing libraries we'll need to get and treat data
import yfinance as yf
import pandas as pd

In [2]:
# creating the list of tickers
idiv = ['ABCB4.SA', 'BRSR6.SA', 'BBSE3.SA', 'BBDC3.SA', 'BBDC4.SA', 'BRAP4.SA', 'BBAS3.SA', 'AGRO3.SA', 'CCRO3.SA', 'CMIG3.SA', 'CMIG4.SA', 'CESP6.SA', 'CSMG3.SA', 'CPLE3.SA', 'CPLE6.SA', 'CPFE3.SA', 'CYRE3.SA', 'DIRR3.SA', 'ELET3.SA', 'ELET6.SA', 'ENAT3.SA', 'ENBR3.SA', 'EGIE3.SA', 'FESA4.SA', 'ROMI3.SA', 'ITSA4.SA', 'ITUB3.SA', 'ITUB4.SA', 'JHSF3.SA', 'LEVE3.SA', 'MRVE3.SA', 'PETR4.SA', 'PSSA3.SA', 'QUAL3.SA', 'SAPR4.SA', 'SANB11.SA', 'CSNA3.SA', 'SYNE3.SA', 'TAEE11.SA', 'TGMA3.SA', 'VIVT3.SA', 'TRPL4.SA', 'UNIP6.SA', 'VBBR3.SA', 'WIZS3.SA']

In [26]:
# getting the quotes for the list and saving it as a Pandas dataframe (df)
# we will get just the "Adj Close" because that is the one used by Technical Indicators / to build Graphs
dfidiv = yf.download(idiv, start="2021-11-01", end = "2022-01-05")["Adj Close"]
# checking the df
dfidiv.head()

[*********************100%***********************]  45 of 45 completed


,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CESP6.SA,...,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VBBR3.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-01,14.909432,25.360001,28.598818,17.251753,20.263098,21.208147,39.595325,11.177370,11.041284,26.32,...,34.180298,3.673196,6.786234,35.329411,14.959925,23.394859,69.546906,21.633011,45.268936,10.13
2021-11-03,15.529428,26.030001,28.785225,17.209688,20.094633,21.746767,36.609837,11.285506,11.418934,26.65,...,34.503468,3.741038,7.063643,36.039841,14.920582,23.423080,68.602249,22.726593,48.263531,10.03
2021-11-04,15.736094,25.379999,28.108273,16.274912,18.764839,21.775621,36.380184,11.000419,11.418934,26.33,...,33.077705,3.644120,7.043094,35.473415,14.891076,23.036135,67.382446,21.881552,48.302422,9.82
2021-11-05,15.991965,26.629999,28.736172,16.934172,19.690771,21.967985,35.556602,11.059402,11.647511,26.24,...,33.451588,3.673196,7.181798,35.300610,15.205814,22.867334,71.087708,22.478050,48.914955,10.09
2021-11-08,16.021490,27.830000,28.922577,16.599627,19.237656,21.948748,36.918678,10.921775,11.389119,26.50,...,33.325096,3.634428,6.986586,34.974197,15.048445,22.777971,70.885941,22.140036,49.391365,9.56


In [15]:
# checking if we got all quotes for all the tickers on the list
print(len(idiv))
dfidiv.shape

45


(44, 45)

In [16]:
# ok! 45 tickers on the list and 45 columns on the df

In [17]:
# we'll make a copy of this df in order to keep the original if we need to check anything
# the copy will be used to calculate the MA 9 and 21
dfidivb = dfidiv.copy()
dfidivb.head()

,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CESP6.SA,...,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VBBR3.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-01,14.909432,25.360001,28.598818,17.251755,20.263098,21.208147,39.595325,11.177370,11.041284,26.32,...,34.180298,3.673196,6.786234,35.329411,14.959925,23.394861,69.546906,21.633011,45.268936,10.13
2021-11-03,15.529428,26.030001,28.785225,17.209688,20.094631,21.746767,36.609837,11.285506,11.418934,26.65,...,34.503468,3.741038,7.063643,36.039841,14.920582,23.423079,68.602249,22.726593,48.263531,10.03
2021-11-04,15.736094,25.379999,28.108273,16.274912,18.764839,21.775621,36.380184,11.000419,11.418934,26.33,...,33.077702,3.644120,7.043094,35.473415,14.891076,23.036135,67.382446,21.881552,48.302422,9.82
2021-11-05,15.991965,26.629999,28.736172,16.934172,19.690767,21.967985,35.556602,11.059402,11.647511,26.24,...,33.451588,3.673196,7.181798,35.300610,15.205814,22.867334,71.087708,22.478050,48.914955,10.09
2021-11-08,16.021490,27.830000,28.922577,16.599625,19.237656,21.948748,36.918678,10.921775,11.389119,26.50,...,33.325096,3.634428,6.986586,34.974197,15.048445,22.777971,70.885941,22.140036,49.391365,9.56


In [18]:
# now we wil replace the value of every cell with the difference of MA21 minus MA09 (this will generate null values for the first 21 lines)
for (columnName) in dfidivb:
    dfidivb[columnName] = (dfidivb[columnName].rolling(9).mean()) - (dfidivb[columnName].rolling(21).mean())

In [19]:
# checking the new df
dfidivb.head(25)

,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CESP6.SA,...,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VBBR3.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2021-11-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-11-12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
# deleting the null lines
dfidivb = dfidivb.dropna(axis = 0, how ='any')
# and visualizing the df
display(dfidivb)

,ABCB4.SA,AGRO3.SA,BBAS3.SA,BBDC3.SA,BBDC4.SA,BBSE3.SA,BRAP4.SA,BRSR6.SA,CCRO3.SA,CESP6.SA,...,SANB11.SA,SAPR4.SA,SYNE3.SA,TAEE11.SA,TGMA3.SA,TRPL4.SA,UNIP6.SA,VBBR3.SA,VIVT3.SA,WIZS3.SA
Date,,,,,,,,,,,,,,,,,,,,,
2021-12-01,0.246499,-1.170952,0.710046,0.102173,0.143485,-0.749152,1.004213,-0.472180,0.056809,-1.066508,...,-0.286296,-0.010000,-0.225629,-0.438875,-0.932519,-0.073177,6.220851,-0.336595,0.905138,-0.667460
2021-12-02,0.274617,-1.179206,0.935968,0.111243,0.164794,-0.829456,1.479608,-0.413352,0.020535,-1.106190,...,-0.280477,0.001692,-0.210951,-0.275516,-0.951094,0.076928,5.662611,-0.398928,0.436596,-0.608889
2021-12-03,0.346942,-1.101904,1.190734,0.146583,0.212631,-0.832357,1.596258,-0.340325,0.076420,-1.121587,...,-0.246758,0.023537,-0.150936,-0.053640,-0.857415,0.248707,4.638480,-0.343539,0.264982,-0.549841
2021-12-06,0.418642,-1.051905,1.336527,0.103530,0.182005,-0.795105,0.406254,-0.275724,0.174735,-1.083333,...,-0.277072,0.035537,-0.064174,0.180427,-0.691809,0.382044,3.546299,-0.384252,0.148310,-0.497460
2021-12-07,0.433170,-0.983809,1.421763,0.041876,0.121812,-0.764724,-0.871448,-0.245765,0.283685,-1.076825,...,-0.411129,0.046151,0.015493,0.343161,-0.472643,0.474403,3.230216,-0.353796,0.077010,-0.453810
2021-12-08,0.509869,-0.873333,1.351849,-0.046489,0.013299,-0.761823,-2.108978,-0.217677,0.327798,-1.071746,...,-0.618856,0.049690,0.094263,0.436611,-0.374780,0.565659,2.678823,-0.310873,-0.083183,-0.451746
2021-12-09,0.619059,-0.698254,1.350003,-0.104119,-0.045682,-0.699992,-3.281078,-0.173830,0.368441,-1.023175,...,-0.760327,0.051843,0.177437,0.558049,-0.131746,0.648718,2.318538,-0.214455,-0.299706,-0.415238
2021-12-10,0.675138,-0.519841,1.436012,-0.113800,-0.042596,-0.636481,-4.482017,-0.140125,0.441971,-0.968730,...,-0.850213,0.062612,0.261915,0.660439,0.243968,0.711920,2.136839,-0.076850,-0.419466,-0.298889
2021-12-13,0.722157,-0.187936,1.363584,-0.087108,-0.018580,-0.525490,-5.594496,-0.108605,0.425820,-0.915873,...,-0.834306,0.071227,0.361724,0.773089,0.614444,0.771811,2.214876,0.066120,-0.549102,-0.199682


In [27]:
# Creating the messages that will be displayed for the tickers (regarding the 4 cases explained in the introduction of this JNB)
# The messages will be based in the values in the last 2 lines of the df (if we call the day you are checking d, they will be d-1 and d-2)
# The expresison: "d-1 > 0" will be used to represent the value on d-1 is positive, etc.
# i) if d-1 > 0  and  d-2 < 0 : BUY
# ii) if d-1 < 0  and  d-2 <0 : STOP
# iii) if d-1 > 0  and  d-2 > 0 : HOLD
# iv) if d-1 < 0  and  d-2 < 0 : WAIT

In [22]:
# creating functions for colored messages to be used in the JNB
def prRed(skk): print("\033[91m {}\033[00m" .format(skk))
def prGreen(skk): print("\033[92m {}\033[00m" .format(skk))
def prYellow(skk): print("\033[93m {}\033[00m" .format(skk))
def prCyan(skk): print("\033[96m {}\033[00m" .format(skk))
def prLightGray(skk): print("\033[97m {}\033[00m" .format(skk))

In [23]:
# creating the function to classify the tickers and print the messages
def alertacolor (acao):
    if (dfidivb.at[dfidivb.index[-1],acao] > 0) and (dfidivb.at[dfidivb.index[-2],acao] < 0):
        prGreen('  Buy!  : ) ')
    elif (dfidivb.at[dfidivb.index[-1],acao] < 0) and (dfidivb.at[dfidivb.index[-2],acao] > 0):
        prRed('  Stoooop!!')
    elif (dfidivb.at[dfidivb.index[-1],acao] > 0) and (dfidivb.at[dfidivb.index[-2],acao] > 0):
        prGreen('  Keep Calm and... Hold  : )')
    else:
        prYellow('  Wait...')

In [24]:
# running this function for the list of tickers
for papel in idiv:
    prCyan (papel+': ') 
    alertacolor(papel)
    print(' ')

 ABCB4.SA: 
   Wait...
 
 BRSR6.SA: 
   Wait...
 
 BBSE3.SA: 
   Wait...
 
 BBDC3.SA: 
   Wait...
 
 BBDC4.SA: 
   Wait...
 
 BRAP4.SA: 
   Buy!  : ) 
 
 BBAS3.SA: 
   Wait...
 
 AGRO3.SA: 
   Keep Calm and... Hold  : )
 
 CCRO3.SA: 
   Wait...
 
 CMIG3.SA: 
   Keep Calm and... Hold  : )
 
 CMIG4.SA: 
   Wait...
 
 CESP6.SA: 
   Wait...
 
 CSMG3.SA: 
   Wait...
 
 CPLE3.SA: 
   Keep Calm and... Hold  : )
 
 CPLE6.SA: 
   Keep Calm and... Hold  : )
 
 CPFE3.SA: 
   Wait...
 
 CYRE3.SA: 
   Wait...
 
 DIRR3.SA: 
   Keep Calm and... Hold  : )
 
 ELET3.SA: 
   Wait...
 
 ELET6.SA: 
   Wait...
 
 ENAT3.SA: 
   Wait...
 
 ENBR3.SA: 
   Wait...
 
 EGIE3.SA: 
   Wait...
 
 FESA4.SA: 
   Stoooop!!
 
 ROMI3.SA: 
   Stoooop!!
 
 ITSA4.SA: 
   Wait...
 
 ITUB3.SA: 
   Wait...
 
 ITUB4.SA: 
   Wait...
 
 JHSF3.SA: 
   Wait...
 
 LEVE3.SA: 
   Wait...
 
 MRVE3.SA: 
   Wait...
 
 PETR4.SA: 
   Wait...
 
 PSSA3.SA: 
   Wait...
 
 QUAL3.SA: 
   Wait...
 
 SAPR4.SA: 
   Wait...
 
 SANB11.SA: 
   Wait...

<div class="alert-info">
    ok!
</div>

<div class="alert-warning">
</br>
  <strong> References </strong>
</div>

###### Youtube Video (in Portuguese): "PYTHON PARA INVESTIMENTOS #4: Simulando uma CARTEIRA DE AÇÕES e comparando com o IBOVESPA"
https://youtu.be/TiNLwmLN-iE?list=PLCAhGm8nJ9CBn51o0x3j1p1LuMRqpeqCy

https://www.geeksforgeeks.org/print-colors-python-terminal/

###### last update: 22.feb.22

###### to go back to my GitHub Page:  https://rafsz.github.io/